## Do these things just once
Calling the training dataset, slicing it and getting the variant's identificators. Variant identificators are composed of \[Chrom, Pos, Ref, Alt\] columns, and we also include the label y. We then add the ID column which corresponds to the unique identifier for that variant (row number). 

In [1]:
import dask.dataframe as dd
import dask.array as da

In [6]:
chrom = 2
cadd = '/s/project/kipoi-cadd/data/raw/v1.3/training_data/training_data.tsv'
cadd_df = dd.read_csv(cadd, sep='\t', dtype={
    'Chrom': 'object',
    'PolyPhenVal': 'float64',
    'SIFTval': 'float64',
    'mirSVR-E': 'float64',
    'mirSVR-Score': 'float64',
    'relCDSpos': 'float64',
    'relProtPos': 'float64',
    'bStatistic': 'float64',
    'isTv': 'float64'})
cadd_df.head()

,y,Chrom,Pos,Ref,Alt,Type,Length,isTv,Consequence,GC,...,SIFTcat,SIFTval,mirSVR-Score.na,targetScan.na,cDNApos.na,CDSpos.na,protPos.na,Grantham.na,PolyPhenVal.na,SIFTval.na
0,0,1,379177,T,G,SNV,0,1.0,NC,0.48,...,UD,0.0,1,1,0,1,1,1,1,1
1,0,1,379274,C,G,SNV,0,1.0,NC,0.54,...,UD,0.0,1,1,0,1,1,1,1,1
2,0,1,379476,A,T,SNV,0,1.0,UP,0.63,...,UD,0.0,1,1,1,1,1,1,1,1
3,0,1,379631,C,G,SNV,0,1.0,UP,0.34,...,UD,0.0,1,1,1,1,1,1,1,1
4,0,1,379724,A,G,SNV,0,0.0,UP,0.28,...,UD,0.0,1,1,1,1,1,1,1,1


In [9]:
num_vars = cadd_df.shape[0].compute()
ident = cadd_df.loc[:,["y", "Chrom", "Pos", "Ref", "Alt"]]
ident['ID'] = dd.from_dask_array(ident.index.values + 1)
ident.set_index(ident.ID, drop=True)
ident.head()

In [17]:
all_variants = '/s/project/kipoi-cadd/data/processed/kipoi/all_variants'
ident.to_parquet(all_variants, compression="uncompressed")

In [33]:
cadd_variants = '/s/project/kipoi-cadd/data/processed/kipoi/chr'+str(chrom)+'/cadd_variant_labels'
ident = cadd_df[cadd_df.Chrom == str(chrom)]
ident.to_parquet(cadd_variants, compression="uncompressed")